In [ ]:
import geopandas
import matplotlib.pyplot as pyplot
# from scipy import interpolate
import numpy as np
# %matplotlib inline

Error: Jupyter notebook failed to launch. 
Error: Jupyter server crashed. Unable to connect. 
Error code from jupyter: 0
Failed to run jupyter as observable with args notebook --no-browser --notebook-dir=C:\Users\Mazel\AppData\Local\Temp\fc2d076b-4cce-4a1f-9e1e-e80196f254df --config=C:\Users\Mazel\AppData\Local\Temp\fc2d076b-4cce-4a1f-9e1e-e80196f254df\jupyter_notebook_config.py --NotebookApp.iopub_data_rate_limit=10000000000.0


In [ ]:
berlinDistricts = geopandas.read_file("../shared/berlinDistricts.geojson")
measurements = geopandas.read_file("data/data_2020-02-20T11-01-00.geojson")
berlinDistricts = berlinDistricts.to_crs(epsg=3068)
measurements = measurements.to_crs(epsg=3068)

pyplot.rcParams['figure.figsize'] = 30, 20
pyplot.rcParams['font.size'] = 20
pyplot.rcParams['axes.titlesize'] = 50
pyplot.rcParams['axes.titlepad'] = 80

berlinPlot = berlinDistricts.boundary.plot(edgecolor="black")
berlinPlot.set_title("Berlin Districts with Measurements")

measurements.plot(ax=berlinPlot, column="value", legend=True)


Error: Jupyter notebook failed to launch. 
Error: Jupyter server crashed. Unable to connect. 
Error code from jupyter: 0
Failed to run jupyter as observable with args notebook --no-browser --notebook-dir=C:\Users\Mazel\AppData\Local\Temp\71421e85-8ad8-441b-b434-86737db70a01 --config=C:\Users\Mazel\AppData\Local\Temp\71421e85-8ad8-441b-b434-86737db70a01\jupyter_notebook_config.py --NotebookApp.iopub_data_rate_limit=10000000000.0


In [ ]:
berlinPlot.figure.savefig("measurements-in-berlin.png")

Error: Jupyter notebook failed to launch. 
Error: Jupyter server crashed. Unable to connect. 
Error code from jupyter: 0
Failed to run jupyter as observable with args notebook --no-browser --notebook-dir=C:\Users\Mazel\AppData\Local\Temp\71421e85-8ad8-441b-b434-86737db70a01 --config=C:\Users\Mazel\AppData\Local\Temp\71421e85-8ad8-441b-b434-86737db70a01\jupyter_notebook_config.py --NotebookApp.iopub_data_rate_limit=10000000000.0


In [ ]:
x = measurements.geometry.x
y = measurements.geometry.y
values = measurements.value

xmin, ymin, xmax, ymax = measurements.total_bounds
xnew = np.linspace(xmin, xmax, int((xmax-xmin)/100))
ynew = np.linspace(ymin, ymax, int((ymax-ymin)/100))

interpolator = interpolate.interp2d(x, y, values, kind="cubic")

valuesnew = interpolator(xnew, ynew)

type(xnew)
# valuesnew



Error: Jupyter notebook failed to launch. 
Error: Jupyter server crashed. Unable to connect. 
Error code from jupyter: 0
Failed to run jupyter as observable with args notebook --no-browser --notebook-dir=C:\Users\Mazel\AppData\Local\Temp\71421e85-8ad8-441b-b434-86737db70a01 --config=C:\Users\Mazel\AppData\Local\Temp\71421e85-8ad8-441b-b434-86737db70a01\jupyter_notebook_config.py --NotebookApp.iopub_data_rate_limit=10000000000.0
